In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_absolute_error
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
import shap

In [2]:
# Especificar ruta al repo clonado
os.chdir("/home/cesar/Documentos/banorte_test")
# Lectura de la informacion
df_final = pd.read_csv('./data/df_final.csv')
df_final = df_final.drop(['Unnamed: 0'],axis=1)

In [ ]:
# Split para complementar con un modelo supervisado
X = df_final.drop(['Rating'], axis=1)
y = df_final['Rating']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# Ajuste
model = XGBRegressor(booster = 'gbtree')
model.fit(X_train, y_train)

In [ ]:
# Pronostico
y_pred = model.predict(X_test)
y_pred_rounded = np.round(y_pred)
# Limitar los valores mayores a 5 a exactamente 5
y_pred_rounded = np.clip(y_pred_rounded, None, 5)
mae = mean_absolute_error(y_test, y_pred_rounded)
print(f"Mean Absolute Error (MAE): {mae}")

In [ ]:
# Calcular SHAP values
explainer = shap.Explainer(model, X_train)
shap_values = explainer(X_test)

plt.figure()
shap.summary_plot(shap_values, X_test)
plt.savefig('figures/shap.png')
plt.show()
plt.close()